# **Import necessary packages**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Import original data and preprocessed data**

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

In [ ]:
sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# **User-defined functions to calculate Metrics and Croston algorithm**

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [ ]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# **Set value for parameters**

In [ ]:
list_params = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
list_params

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 50
validation_training_ratio = 0.95

# Pipeline for Croston

In [ ]:
def Croston_output_all_params(sales_pattern, list_params, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in range(len(train_second)):
                yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                            'Actual Data': train_second,
                                            'Forecast': predictions,
                                            'Alpha': [i for count in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

pattern_df_result_all_params = Croston_output_all_params(sales_smooth,
                                                      list_params,
                                                      start_train_date,
                                                      end_train_date, 
                                                      n_pred_days,
                                                      num_train_needed,
                                                      validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 1.7848220968166666 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOUSEHOLD_1_007_CA_1_validation,6.0,2.250400,0.1
1,HOUSEHOLD_1_007_CA_1_validation,2.0,2.506399,0.1
2,HOUSEHOLD_1_007_CA_1_validation,0.0,2.470693,0.1
3,HOUSEHOLD_1_007_CA_1_validation,2.0,2.470693,0.1
4,HOUSEHOLD_1_007_CA_1_validation,3.0,2.271466,0.1
...,...,...,...,...
128533,HOUSEHOLD_2_176_CA_4_validation,2.0,0.013514,0.9
128534,HOUSEHOLD_2_176_CA_4_validation,3.0,0.122581,0.9
128535,HOUSEHOLD_2_176_CA_4_validation,3.0,1.179592,0.9
128536,HOUSEHOLD_2_176_CA_4_validation,2.0,2.610480,0.9


In [ ]:
pattern_df_result_all_params.to_csv('Croston_Smooth_All_Parameters_Hyperparameter_Tuning_Data.csv')

# **Summary all metrics**

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.15787853676666638 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_003_CA_3_validation,0.1,0.706240,86.8183,107.8808,37.4037,1.196685,1.415761
1,HOUSEHOLD_1_003_CA_3_validation,0.2,0.722277,88.7897,109.0314,38.5370,1.223858,1.445969
2,HOUSEHOLD_1_003_CA_3_validation,0.3,0.737034,90.6039,109.4451,38.8310,1.248864,1.480444
3,HOUSEHOLD_1_003_CA_3_validation,0.4,0.750393,92.2460,109.4112,38.4836,1.271499,1.520085
4,HOUSEHOLD_1_003_CA_3_validation,0.5,0.763477,93.8545,109.0891,37.6800,1.293670,1.568190
...,...,...,...,...,...,...,...,...
3469,HOUSEHOLD_2_513_CA_3_validation,0.5,1.296308,121.7462,153.6388,77.9163,1.908454,2.699069
3470,HOUSEHOLD_2_513_CA_3_validation,0.6,1.258065,118.1545,154.6965,85.4172,1.852152,2.683092
3471,HOUSEHOLD_2_513_CA_3_validation,0.7,1.229784,115.4984,155.3461,92.6602,1.810516,2.690504
3472,HOUSEHOLD_2_513_CA_3_validation,0.8,1.240724,116.5259,156.1753,103.4166,1.826622,2.758835


# **Check what products has unexpected metrics result**

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
36,HOUSEHOLD_1_018_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
37,HOUSEHOLD_1_018_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
38,HOUSEHOLD_1_018_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
39,HOUSEHOLD_1_018_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
40,HOUSEHOLD_1_018_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
3451,HOUSEHOLD_2_490_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3452,HOUSEHOLD_2_490_CA_3_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3453,HOUSEHOLD_2_490_CA_3_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3454,HOUSEHOLD_2_490_CA_3_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
36,HOUSEHOLD_1_018_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
37,HOUSEHOLD_1_018_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
38,HOUSEHOLD_1_018_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
39,HOUSEHOLD_1_018_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
40,HOUSEHOLD_1_018_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
3451,HOUSEHOLD_2_490_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3452,HOUSEHOLD_2_490_CA_3_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3453,HOUSEHOLD_2_490_CA_3_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3454,HOUSEHOLD_2_490_CA_3_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
36,HOUSEHOLD_1_018_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
37,HOUSEHOLD_1_018_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
38,HOUSEHOLD_1_018_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
39,HOUSEHOLD_1_018_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
40,HOUSEHOLD_1_018_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
3451,HOUSEHOLD_2_490_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3452,HOUSEHOLD_2_490_CA_3_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3453,HOUSEHOLD_2_490_CA_3_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3454,HOUSEHOLD_2_490_CA_3_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
36,HOUSEHOLD_1_018_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
37,HOUSEHOLD_1_018_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
38,HOUSEHOLD_1_018_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
39,HOUSEHOLD_1_018_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
40,HOUSEHOLD_1_018_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
3451,HOUSEHOLD_2_490_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3452,HOUSEHOLD_2_490_CA_3_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3453,HOUSEHOLD_2_490_CA_3_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3454,HOUSEHOLD_2_490_CA_3_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# **Metrics Statistics**

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 19.94818652849741 %
Percentage of unexpected values of WMAPE is: 19.94818652849741 %
Percentage of unexpected values of SMAPE is: 19.689119170984455 %
Percentage of unexpected values of MAPE is: 19.94818652849741 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


# Filter all rows that have unexpected metrics values 

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

# Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,1.083846,118.351756,81.707309,61.558830,2.821612,3.508439
0.2,1.047948,109.179733,82.483772,63.403137,2.843344,3.552696
0.3,1.027763,102.941569,83.409815,65.206059,2.882406,3.613109
0.4,1.015783,98.189195,84.380991,67.137729,2.932308,3.681687
0.5,1.012626,95.121044,85.388749,69.343526,2.987714,3.755560
0.6,1.016366,93.349910,86.474675,71.655960,3.047351,3.834888
0.7,1.026028,92.624377,87.681316,74.111271,3.112081,3.921940
0.8,1.041600,92.796843,89.067481,76.780888,3.185144,4.020593
0.9,1.063057,93.751260,90.631435,79.685595,3.266997,4.137544


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.5
The optimum Alpha based on Mean WMAPE is: 0.7
The optimum Alpha based on Mean SMAPE is: 0.1
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.1
The optimum Alpha based on Mean RMSE is: 0.1


# **Get MEDIAN metrics value of each Alpha**

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.801066,62.63710,73.24605,58.00175,1.938897,2.393224
0.2,0.815852,64.30045,74.64260,60.57490,1.989995,2.463046
0.3,0.834762,65.92810,75.51665,62.67535,2.049281,2.534141
0.4,0.859525,67.30745,76.52285,64.55685,2.101829,2.593380
0.5,0.876152,68.27270,77.61170,66.76575,2.153192,2.632859
0.6,0.890359,70.79340,78.54410,68.89540,2.205022,2.713830
0.7,0.914475,72.45505,80.21925,71.98930,2.260365,2.822989
0.8,0.941018,74.09055,82.59615,73.72405,2.318711,2.902326
0.9,0.964709,76.32815,83.73315,75.85795,2.386331,2.998567


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.1
The optimum Alpha based on Median of WMAPE is: 0.1
The optimum Alpha based on Median of SMAPE is: 0.1
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.1
The optimum Alpha based on Median of RMSE is: 0.1


# **Run Croston model after decide best paramaters**

In [ ]:
best_alpha = 0.1

In [ ]:
def Croston_test_data(best_alpha, sales_pattern, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = Croston_test_data(best_alpha, sales_smooth, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0724075616333342 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_003_CA_3_validation,0.908132,156.4781,145.1266,39.5589,1.117701,1.172827
1,HOUSEHOLD_1_004_CA_3_validation,0.815354,63.7034,75.7586,51.5005,2.320623,3.043270
2,HOUSEHOLD_1_007_CA_1_validation,0.987392,47.8505,49.1948,48.3289,0.683579,0.866660
3,HOUSEHOLD_1_007_CA_3_validation,0.731363,88.3091,100.5915,68.0634,2.081571,2.422516
4,HOUSEHOLD_1_018_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...
381,HOUSEHOLD_2_483_CA_3_validation,0.633350,56.8391,58.2517,59.6317,1.948769,2.690427
382,HOUSEHOLD_2_490_CA_2_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
383,HOUSEHOLD_2_490_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
384,HOUSEHOLD_2_510_CA_3_validation,0.708063,44.8547,50.7204,45.4414,1.089328,1.312952


In [ ]:
df_result_final.isnull().sum()

Product     0
MASE       73
WMAPE      73
SMAPE      74
MAPE       77
MAE         0
RMSE        0
dtype: int64

In [ ]:
df_result_final.to_csv('Croston_Smooth_Test_Data.csv')